# IBM Applied Data Science - Capstone project
## Exercises

### Topic: Predict Car Accident Severity in Seattle, USA

In [1]:
%matplotlib inline

# import libraries
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

import os.path

In [2]:
# get the data

data_url = 'https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv'

df = pd.read_csv(data_url)

/Users/martin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# To be run independently
df.info()
#df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEVERITYCODE    194673 non-null  int64  
 1   X               189339 non-null  float64
 2   Y               189339 non-null  float64
 3   OBJECTID        194673 non-null  int64  
 4   INCKEY          194673 non-null  int64  
 5   COLDETKEY       194673 non-null  int64  
 6   REPORTNO        194673 non-null  object 
 7   STATUS          194673 non-null  object 
 8   ADDRTYPE        192747 non-null  object 
 9   INTKEY          65070 non-null   float64
 10  LOCATION        191996 non-null  object 
 11  EXCEPTRSNCODE   84811 non-null   object 
 12  EXCEPTRSNDESC   5638 non-null    object 
 13  SEVERITYCODE.1  194673 non-null  int64  
 14  SEVERITYDESC    194673 non-null  object 
 15  COLLISIONTYPE   189769 non-null  object 
 16  PERSONCOUNT     194673 non-null  int64  
 17  PEDCOUNT  

In [4]:
# count unique values for a given column

id="WEATHER"

td = df[id].value_counts().to_frame()

#td.replace('999', np.nan, inplace = True)
#td.loc[('999'),:]
td

,WEATHER
Clear,111135
Raining,33145
Overcast,27714
Unknown,15091
Snowing,907
Other,832
Fog/Smog/Smoke,569
Sleet/Hail/Freezing Rain,113
Blowing Sand/Dirt,56
Severe Crosswind,25


In [31]:
# count number of NaNs

def df_qc(df):

    missing_data = df.isnull()
    df_nan = pd.DataFrame(columns=['ID', 'val', 'nan'])

    for column in missing_data.columns.values.tolist():
        try:
            val_ok = missing_data[column].value_counts()[0]
            val_nan = missing_data[column].value_counts()[1]

            df_new_row = pd.DataFrame(data=np.array([[column,val_ok,val_nan]]), columns=['ID','val','nan'])
            df_nan = pd.concat([df_nan,df_new_row], ignore_index=True)

        except:
            val_ok = missing_data[column].value_counts()[0]
            df_new_row = pd.DataFrame(data=np.array([[column,val_ok,0]]), columns=['ID','val','nan'])
            df_nan = pd.concat([df_nan,df_new_row], ignore_index=True)

    # cast values as integer
    df_nan = df_nan.astype({'val' :'int','nan':'int'})

    df_nan.set_index('ID', inplace=True)
    df_nan.sort_values(['nan'], ascending=True, axis=0, inplace=True)

    return df_nan

    

In [33]:
df_qc(df_c)

,val,nan
ID,,
SEVERITYCODE,194673,0
YEAR,194673,0
HITPARKEDCAR,194673,0
CROSSWALKKEY,194673,0
SEGLANEKEY,194673,0
SDOT_COLDESC,194673,0
SDOT_COLCODE,194673,0
INCDTTM,194673,0
MONTH,194673,0


## Clean-up Data and Format

Target value: **SEVERITYCODE**
Goal: Predict severity of accident

* INJURIES: total number of injuries
* SERIOUSINJURIES: total number of serious injuries

* COLLISIONTYPE: how did the cars collide
---
### Participants
* PERSONCOUNT: How many people were involved 
* PEDCOUNT: Number of pedestrians involved
* PEDCYLCOUNT: number of bikes involved
* VEHCOUNT: number of vehicles involved
---
### Nature
* LIGHTCOND / WEATHER / ROADCOND
---
* INCDATE: date of accident
---
### Location
* X / Y: where accident has happened
* LOCATION: exact location

* JUNCTIONTYPE
* CROSSWALKKEY
* SEGLANEKEY: Lane segment
* HITPARKEDCAR
* ADDRTYPE
---
### Fault
* INATTENTIONIND: inattentive or not
* UNDERINFL: influence of narcotics
* PEDROWNOTGRNT: pedestrian right granted?
* SPEEDING
---
### Description
* ST_COLCODE: Description for the accident


# to do
* create number in cars
* create number values for objects 
* convert: UNDERINFL
* create categories
    * years
    * weather and light

* normalize data
* xplot severitydesc and severitycode
* correlation heat map



# Format Data

In [418]:
## 1) Extract Year and Month and create new column

df_c = df.copy()
df_c["YEAR"] = int(df["INCDATE"][0][0:4])
df_c["MONTH"] = int(df["INCDATE"][0][5:7])

## 2) Create from PERSONCOUNT / PEDCOUNT / PEDCYLCOUNT the number of people in cars
# int

df_c["PCAR"] = df["PERSONCOUNT"] - df["PEDCOUNT"] - df["PEDCYLCOUNT"] 
df_c.rename(columns={'PEDCOUNT':'PPED', 'PEDCYLCOUNT':'PCYC'}, inplace=True)

df_c.drop('PERSONCOUNT', inplace= True, axis=1)


In [419]:
## 3) convert value into integer

#id="WEATHER"

# make sure no NaN values
#temp_df = df_c[id].fillna(0)


#tempd2 = pd.DataFrame(pd.factorize(temp_df)[0])

#out = pd.concat([df_c[id], tempd2], axis=1, sort=False)
#out["WEATHER"]

In [470]:
# Function to condition column values

def cond_c(df_raw2, column_names, kill_nan=False, replace_nan=["no"], int_trans=False, verbose=True, d_info=True):
    
    df_raw = df_raw2.copy()
    
    '''
    clean df values and transform
    INPUT:
    df_raw: full data frame
    column_names: LIST of column names
    kill_nan=False: remove NaN from data frame (True/False)
    replace_nan="no": LIST if kill_nan == False, possibility to replace with value or mean/median/str/value. 
                      replace_nan=no: do not touch; leave unchanged
    int_trans=True: convert uniques obj into integer. add new column called column_names[n]_INT and rename old one 
                    with prefix "_"
    verbose=True: output info of nans and d_info: number of unique values
    d_info=True: number of unique values
    '''
 

    for n,i in enumerate(column_names):
        # get number of missing entries
        missing_data = df_raw[column_names[n]].isna().sum()
        num_rows = df_raw.shape[0]
        perc_nan = round(missing_data / num_rows * 100,2)
    
# output column info if verbose True
        if verbose:
            print(f"\n{column_names[n]}")
            print("\r---------------------------------------------------")
            print(f"Number of missing entries: {missing_data} ({perc_nan} % of entries)")
            print("\r---------------------------------------------------")

            if d_info:
                print(df_raw[column_names[n]].value_counts())
                print("")
            else:
                pass
        else:
            pass
        
        
# kill rows in whole data frame if nan value
        if kill_nan:
            df_raw = df_raw[df_raw[column_names[n]].notna()]  
            int_flag = True
        else:
            pass

       
    
# fill NaN rows in df. 
# the if loop checks the type of the values and allow to replace them eventually with calculated median/mean
# or provided str/int/float

        # don't do anything
        if replace_nan[n] == "no":
            int_flag = False
            
            if verbose:
                print(f">> {column_names[n]} nothing replaced.")

        # replace NaN with median
        elif replace_nan[n] == "median" and (df_raw[column_names[n]].dtypes == "float64" or df_raw[column_names[n]].dtypes == "int64"):
 
            av = df_raw[column_names[n]].median()
            df_raw[column_names[n]] = df_raw[column_names[n]].replace(np.nan, av)
            if verbose:
                print(f">> {column_names[n]} NaN replaced with median ({missing_data} values).")
            int_flag = False 

        # replace NaN with mean
        elif replace_nan[n] == "mean" and (df_raw[column_names[n]].dtypes == "float64" or df_raw[column_names[n]].dtypes == "int64"):
            
            av = df_raw[column_names[n]].mean()
            df_raw[column_names[n]] = df_raw[column_names[n]].replace(np.nan, av)
            
            if verbose:
                print(f">> {column_names[n]} NaN replaced with mean ({missing_data} values).")
            int_flag = False 
            
        # replace NaN with provided float or integer                                               
        elif (isinstance(replace_nan[n], int) or isinstance(replace_nan[n], float)) and (df_raw[column_names[n]].dtypes == "float64" or df_raw[column_names[n]].dtypes == "int64"):

            df_raw[column_names[n]] = df_raw[column_names[n]].replace(np.nan, replace_nan[n])
            
            if verbose:
                print(f">> {column_names[n]} NaN replaced with \"{replace_nan[n]}\" ({missing_data} values).")
            int_flag = False
        
        # replace NaN with provided string
        elif isinstance(replace_nan[n], str) and df_raw[column_names[n]].dtypes == "O":
            
            df_raw[column_names[n]] = df_raw[column_names[n]].replace(np.nan, replace_nan[n])
            
            if verbose:
                print(f">> {column_names[n]} NaN replaced with \"{replace_nan[n]}\" ({missing_data} values).")
            int_flag = True
        
        # if type does not match, pass.
        else:
            if verbose:
                print(f"!! replace_nan = {replace_nan[n]} does not fit column type {df_raw[column_names[n]].dtypes}. pass")
            int_flag = False
            pass

                
            
# replace obj-type entries as unique integers and add new column to df with name column_names[n]_INT
    
        if int_trans and int_flag and df_raw[column_names[n]].dtypes == "O":
            temp = pd.DataFrame(pd.factorize(df_raw[column_names[n]])[0])
            temp.columns = [(column_names[n] + "_INT")]
            df_raw = pd.concat([df_raw, temp], axis=1, sort=False)

            df_raw.rename(columns={column_names[n] : ('_' + column_names[n])}, inplace=True)
            
            if verbose:
                print(f">> Unique column values counted and added.")
                print(f"\t-- New column created of type <int> \"{column_names[n]}_INT\" from unique values in \"{column_names[n]}\"")
                print(f"\t-- Column ID \"{column_names[n]}\" renamed to \"_{column_names[n]}\"")
            

        else:
            pass
        
        
        
    return df_raw
    
    
    

    

In [477]:
out = cond_c(df_c,column_names = ["LOCATION"], kill_nan=False, replace_nan=["Unknown"], int_trans = True, verbose=True,d_info=True)
cond_c(out,column_names = ["_LOCATION"])
#cond_c(d,column_names = ["WEATHER"], kill_nan=False, verbose=True,d_info=True)


LOCATION
---------------------------------------------------
Number of missing entries: 2677 (1.38 % of entries)
---------------------------------------------------
BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB AND AURORA AVE N    276
BATTERY ST TUNNEL SB BETWEEN AURORA AVE N AND ALASKAN WY VI SB    271
N NORTHGATE WAY BETWEEN MERIDIAN AVE N AND CORLISS AVE N          265
AURORA AVE N BETWEEN N 117TH PL AND N 125TH ST                    254
6TH AVE AND JAMES ST                                              252
                                                                 ... 
30TH AVE SW AND SW ADAMS ST                                         1
ERSKINE WAY SW BETWEEN SW HUDSON N ST AND SW HUDSON S ST            1
E HOWELL ST BETWEEN 33RD AVE AND 34TH AVE                           1
39TH AVE NE BETWEEN NE 73RD ST AND NE 75TH ST                       1
NE 36TH ST BETWEEN 47TH AVE NE AND DEAD END                         1
Name: LOCATION, Length: 24102, dtype: int64

>> LOCATION NaN rep

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,YEAR,MONTH,PCAR,LOCATION_INT
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,NaN,10,Entering at angle,0,0,N,2013,3,2,0
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,NaN,11,From same direction - both going straight - bo...,0,0,N,2013,3,2,1
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,NaN,32,One parked--one moving,0,0,N,2013,3,4,2
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,NaN,23,From same direction - all others,0,0,N,2013,3,3,3
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,NaN,10,Entering at angle,0,0,N,2013,3,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194668,2,-122.290826,47.565408,219543,309534,310814,E871089,Matched,Block,NaN,...,NaN,24,From opposite direction - both moving - head-on,0,0,N,2013,3,3,11652
194669,1,-122.344526,47.690924,219544,309085,310365,E876731,Matched,Block,NaN,...,NaN,13,From same direction - both going straight - bo...,0,0,N,2013,3,2,5377
194670,2,-122.306689,47.683047,219545,311280,312640,3809984,Matched,Intersection,24760.0,...,NaN,28,From opposite direction - one left turn - one ...,0,0,N,2013,3,3,1327
194671,2,-122.355317,47.678734,219546,309514,310794,3810083,Matched,Intersection,24349.0,...,NaN,5,Vehicle Strikes Pedalcyclist,4308,0,N,2013,3,1,19913


In [411]:
d[["WEATHER_INT","_WEATHER"]].head(20)

,WEATHER_INT,_WEATHER
0,0,Overcast
1,1,Raining
2,0,Overcast
3,2,Clear
4,1,Raining
5,2,Clear
6,1,Raining
7,2,Clear
8,2,Clear
9,2,Clear


In [9]:
## 4) convert UNDERINFL into int

In [10]:
## 5) convert 